### Importamos Librerias

In [1]:
import pandas as pd

### Cargamos el Archivo Business

In [2]:
df_business = pd.read_pickle('Yelp/business.pkl')
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Encontramos un error dentro de los datos, ya que cuenta con columnas duplicadas las cuales seran eliminadas

In [3]:
df_business = df_business.iloc[:,0:14]
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


- Contamos los nulos en codigo_postal

In [4]:
df_business['postal_code'].isnull().any()

False

Tras investigacion determinamos que el codigo postal en EEUU es exclusivamente numerico, mientras que se encontraron valores no numericos entre los codigos. <br>
Forzamos conversion de codigo_postal a valor numerico, convirtiendo no numericos a NaN.

In [5]:
df_business['postal_code'] = pd.to_numeric(df_business['postal_code'], errors='coerce')

total = df_business['postal_code'].size
non_nan = df_business['postal_code'].count()

df_business = df_business.dropna(subset=['postal_code'])

df_business['postal_code'] = df_business['postal_code'].astype(int)

print(f'Se eliminaron {total - non_nan} codigos postales no numericos')

Se eliminaron 5629 codigos postales no numericos


Confirmamos que la columna postal_code se encuentre en formato ``int32``

In [6]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 144717 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   144717 non-null  object
 1   name          144717 non-null  object
 2   address       144717 non-null  object
 3   city          144717 non-null  object
 4   state         144714 non-null  object
 5   postal_code   144717 non-null  int32 
 6   latitude      144717 non-null  object
 7   longitude     144717 non-null  object
 8   stars         144717 non-null  object
 9   review_count  144717 non-null  object
 10  is_open       144717 non-null  object
 11  attributes    131588 non-null  object
 12  categories    144615 non-null  object
 13  hours         122558 non-null  object
dtypes: int32(1), object(13)
memory usage: 16.0+ MB


### Creamos un diccionario de *codigos postales*

In [7]:
postal_codes = {
    'Alabama': [('31905', '31905'), ('35001', '36999'), ('38852', '38852')],
    'Alaska': [('98791', '98791'), ('99501', '99999')],
    'Arizona': [('84531', '84531'), ('84536', '84536'), ('85001', '86599'), ('86631', '86631'), ('87328', '87328'), ('89024', '89024')],
    'Arkansas': [('38041', '38041'), ('38063', '38063'), ('38769', '38769'), ('65729', '65729'), ('65733', '65733'), ('65761', '65761'), ('71601', '72999'), ('75502', '75502'), ('75556', '75556'), ('78081', '78081')],
    'California': [('89010', '89010'), ('89019', '89019'), ('89060', '89060'), ('89439', '89439'), ('90001', '96199'), ('96222', '96222'), ('97635', '97635')],
    'Colorado': [('80000', '81699'), ('82063', '82063')],
    'Connecticut': [('06001', '06999')],
    'Delaware': [('19700', '19999')],
    'District of Columbia': [('20001', '20599'), ('56901', '56999')],
    'Florida': [('32001', '34999')],
    'Georgia': [('30001', '31999'), ('32350', '32350'), ('39801', '39899'), ('39901', '39901')],
    'Hawaii': [('96371', '96371'), ('96373', '96373'), ('96385', '96385'), ('96477', '96477'), ('96701', '96899')],
    'Idaho': [('59847', '59847'), ('83111', '83111'), ('83120', '83120'), ('83201', '83899'), ('89832', '89832'), ('97910', '97913'), ('99128', '99128')],
    'Illinois': [('52761', '52761'), ('60001', '62999'), ('63673', '63673')],
    'Indiana': [('46001', '47999')],
    'Iowa': [('50001', '52899'), ('55954', '55954'), ('56027', '56027'), ('56129', '56129'), ('68119', '68120')],
    'Kansas': [('66001', '67999'), ('68325', '68325'), ('68978', '68978')],
    'Kentucky': [('38079', '38079'), ('40001', '42799')],
    'Louisiana': [('70000', '71499'), ('71545', '71545'), ('71749', '71749')],
    'Maine': [('03901', '04999')],
    'Maryland': [('20331', '20331'), ('20588', '20588'), ('20601', '21999')],
    'Massachusetts': [('01001', '02799'), ('5501', '5501'), ('5544', '5544')],
    'Michigan': [('48001', '49999'), ('54554', '54554')],
    'Minnesota': [('51360', '51360'), ('55001', '56799'), ('57026', '57026'), ('57030', '57030'), ('57068', '57068'), ('58225', '58225')],
    'Mississippi': [('38160', '38160'), ('38514', '38514'), ('38597', '38597'), ('38601', '39799'), ('71233', '71233')],
    'Missouri': [('51630', '51630'), ('51640', '51640'), ('52542', '52542'), ('52573', '52573'), ('52626', '52626'), ('63001', '65899'), ('72644', '72644')],
    'Montana': [('57724', '57724'), ('59000', '59999'), ('82801', '82801')],
    'Nebraska': [('51557', '51557'), ('57078', '57078'), ('66541', '66541'), ('68001', '69399'), ('80758', '80758'), ('82082', '82082')],
    'Nevada': [('84034', '84034'), ('88901', '89999')],
    'New Hampshire': [('00210', '00215'), ('03001', '03899')],
    'New Jersey': [('07001', '08999'), ('10004', '10004')],
    'New Mexico': [('79837', '79837'), ('79922', '79922'), ('81137', '81137'), ('86504', '86504'), ('86515', '86515'), ('87001', '88499'), ('88603', '88603')],
    'New York': [('501', '501'), ('544', '544'), ('6390', '6390'), ('10001', '14999')],
    'North Carolina': [('27001', '28999')],
    'North Dakota': [('56701', '56799'), ('57638', '57638'), ('57641', '57642'), ('57645', '57645'), ('57648', '57648'), ('57660', '57660'), ('57683', '57683'), ('58000', '58899'), ('58982', '58982'), ('59221', '59221'), ('59270', '59270'), ('59275', '59275')],
    'Ohio': [('43001', '45999')],
    'Oklahoma': [('67950', '67950'), ('73001', '73199'), ('73367', '73367'), ('73371', '73371'), ('73401', '74999'), ('76356', '76356'), ('79051', '79051')],
    'Oregon': [('89421', '89421'), ('97001', '97999'), ('99362', '99362')],
    'Pennsylvania': [('15001', '19699'), ('19815', '19815'), ('19853', '19853'), ('19925', '19925')],
    'Rhode Island': [('2029', '2029'), ('2789', '2789'), ('02801', '02999')],
    'South Carolina': [('29001', '29999')],
    'South Dakota': [('51001', '51001'), ('51023', '51023'), ('56144', '56144'), ('56164', '56164'), ('56219', '56219'), ('57001', '57799'), ('57840', '57841'), ('57949', '57949'), ('58234', '58234'), ('58439', '58439'), ('58623', '58623'), ('58649', '58649'), ('58653', '58653'), ('68719', '68719'), ('69201', '69201'), ('69212', '69212'), ('69216', '69216'), ('82701', '82701')],
    'Tennessee': [('31788', '31788'), ('37001', '38599'), ('42223', '42223'), ('72338', '72338')],
    'Texas': [('73301', '73301'), ('73344', '73344'), ('73949', '73949'), ('73960', '73960'), ('75001', '79999'), ('88501', '88599')],
    'Utah': [('81324', '81324'), ('82930', '82930'), ('83312', '83312'), ('83342', '83342'), ('84001', '84799'), ('86044', '86044'), ('86514', '86514')],
    'Vermont': [('05001', '05499'), ('05601', '05999')],
    'Virginia': [('20101', '20199'), ('20598', '20598'), ('22001', '24699')],
    'Washington': [('98001', '99499'), ('99536', '99536')],
    'West Virginia': [('24604', '24604'), ('24701', '26899'), ('26905', '26905')],
    'Wisconsin': [('52820', '52820'), ('53001', '54999')],
    'Wyoming': [('57717', '57717'), ('82001', '83199'), ('83414', '83414')]
}


#### Creamos la funcion que asigna el <u>estado</u> dependiendo del <u>codigo postal<u>

In [8]:
def assign_state(df, postal_codes):
    """
    Asigna el estado en el DataFrame basándose en el código postal.
    
    df (pd.DataFrame): DataFrame con una columna 'postal_code'.
    postal_codes (dict): Diccionario que mapea estados a rangos de códigos postales.
    
    Returns:
    pd.DataFrame: DataFrame con una nueva columna 'state' que indica el estado correspondiente.
    """
    def find_state(postal_code):
        for state, ranges in postal_codes.items():
            for start, end in ranges:
                if int(start) <= int(postal_code) <= int(end):
                    return state
        return 'Unknown'
    
    df['state'] = df['postal_code'].apply(find_state)
    
    return df

Corremos la funcion sobre el *dataframe* y checkeamos los resultados

In [9]:
assign_state(df_business, postal_codes)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,California,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,Missouri,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,Arizona,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,Pennsylvania,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,Pennsylvania,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150339,2O2K6SXPWv56amqxCECd4w,The Plum Pit,4405 Pennell Rd,Aston,Pennsylvania,19014,39.856185,-75.427725,4.5,14,1,"{'RestaurantsDelivery': 'False', 'BusinessAcce...","Restaurants, Comfort Food, Food, Food Trucks, ...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,Tennessee,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,Indiana,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,Illinois,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


- Exportamos el archivo como ``Parquet``

In [10]:
import pyarrow as pa
import pyarrow.parquet as pq
parquet_filename = 'Yelp/business.parquet'
table = pa.Table.from_pandas(df_business)
pq.write_table(table, parquet_filename)